In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
import shutil
from tqdm import tqdm
from PIL import Image

DATA_PATH = "../input/"
TRAIN_PATH = DATA_PATH + 'train_images/'

print(os.listdir(DATA_PATH))

from glob import glob 
from skimage.io import imread
import gc

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical


['train.zip', 'test_images', 'sample_submission.csv.zip', 'description.md', 'test_images.zip', 'train_images', 'train_images.zip', 'test.zip', 'sample_submission.csv', 'test.csv.zip', 'train.csv', 'train.csv.zip', 'test.csv', 'aptos2019-blindness-detection']


2025-08-18 20:39:29.615094: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755549569.624585       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755549569.628551       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-18 20:39:29.644392: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
base_tile_dir = DATA_PATH + 'train_images/'

df = pd.DataFrame({'path': glob(base_tile_dir +'/*.png')})

df['id'] = df.path.map(lambda x: x.split('/')[3].split(".")[0])

labels = pd.read_csv(DATA_PATH + "train.csv")
labels = labels.rename(index=str, columns={'id_code':'id', 'diagnosis':'label'})

df_data = df.merge(labels, on = "id")

df_data.head()


,path,id,label
0,../input/train_images/184a185e7447.png,184a185e7447,0
1,../input/train_images/c4aef0d88d1b.png,c4aef0d88d1b,0
2,../input/train_images/d69698f838db.png,d69698f838db,0
3,../input/train_images/15cc2aef772a.png,15cc2aef772a,1
4,../input/train_images/f7edc074f06b.png,f7edc074f06b,2


In [4]:
SAMPLE_SIZE = 150 # load 80k negative examples

# take a random sample of class 0 with size equal to num samples in class 1
df_0 = df_data[df_data['label'] == 0].sample(SAMPLE_SIZE*3, random_state = 101)
# filter out class 1
df_1 = df_data[df_data['label'] == 1].sample(SAMPLE_SIZE*2, random_state = 101)
# filter out class 2
df_2 = df_data[df_data['label'] == 2].sample(SAMPLE_SIZE*2, random_state = 101)
# filter out class 3
df_3 = df_data[df_data['label'] == 3].sample(SAMPLE_SIZE, random_state = 101)
# filter out class 4
df_4 = df_data[df_data['label'] == 4].sample(200, random_state = 101)

# concat the dataframes
df_data = shuffle(pd.concat([df_0, df_1, df_2, df_3, df_4], axis=0).reset_index(drop=True))

print(df_data.head())
print(df_data.label.value_counts())

# train_test_split # stratify=y creates a balanced validation set.
y = df_data['label']
df_train, df_val = train_test_split(df_data, test_size=0.10, random_state=101, stratify=y)

# Create directories
train_path = 'base_dir/train'
valid_path = 'base_dir/valid'
test_path = '../input/test'
for fold in [train_path, valid_path]:
    for subf in ["0", "1","2","3","4"]:
        os.makedirs(os.path.join(fold, subf))


                                        path            id  label
108   ../input/train_images/e0313be77035.png  e0313be77035      0
266   ../input/train_images/44271f3cb18f.png  44271f3cb18f      0
1305  ../input/train_images/0243404e8a00.png  0243404e8a00      4
1049  ../input/train_images/917f76f360b6.png  917f76f360b6      2
973   ../input/train_images/907aaff827e5.png  907aaff827e5      2
label
0    450
2    300
1    300
4    200
3    150
Name: count, dtype: int64


In [5]:
# Set the id as the index in df_data
df_data.set_index('id', inplace=True)
df_data.head()


,path,label
id,,
e0313be77035,../input/train_images/e0313be77035.png,0
44271f3cb18f,../input/train_images/44271f3cb18f.png,0
0243404e8a00,../input/train_images/0243404e8a00.png,4
917f76f360b6,../input/train_images/917f76f360b6.png,2
907aaff827e5,../input/train_images/907aaff827e5.png,2


In [6]:
IMAGE_SIZE = 96

for image in tqdm(df_train['id'].values):
    # the id in the csv file does not have the .tif extension therefore we add it here
    fname = image + '.png'
    label = str(df_data.loc[image,'label']) # get the label for a certain image
    src = os.path.join(TRAIN_PATH, fname)
    dst = os.path.join(train_path, label, fname)
    
    pil_im = Image.open(src)
    resized_image = pil_im.resize((IMAGE_SIZE, IMAGE_SIZE))
    resized_image.save(dst)

for image in tqdm(df_val['id'].values):
    fname = image + '.png'
    label = str(df_data.loc[image,'label']) # get the label for a certain image
    src = os.path.join(TRAIN_PATH, fname)
    dst = os.path.join(valid_path, label, fname)
    
    pil_im = Image.open(src)
    resized_image = pil_im.resize((IMAGE_SIZE, IMAGE_SIZE))
    resized_image.save(dst)


100%|██████████| 140/140 [00:11<00:00, 12.04it/s]


In [7]:
from keras.preprocessing.image import ImageDataGenerator

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 32
val_batch_size = 32

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

datagen = ImageDataGenerator(preprocessing_function=lambda x:(x - x.mean()) / x.std() if x.std() > 0 else x,
                            horizontal_flip=True,
                            vertical_flip=True)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')

# Note: shuffle=False causes the test dataset to not be shuffled
test_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)


ImportError: cannot import name 'ImageDataGenerator' from 'keras.preprocessing.image' (/usr/local/lib/python3.11/dist-packages/keras/api/preprocessing/image/__init__.py)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import RMSprop, Adam

kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.5

model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3)))
model.add(Conv2D(first_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(second_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Conv2D(third_filters, kernel_size, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

#model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(256, use_bias=False))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(5, activation = "sigmoid"))

# Compile the model
model.compile(Adam(0.01), loss = "categorical_crossentropy", metrics=["accuracy"])

print("Done !")


Done !


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-08-18 20:41:39.793921: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [9]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
earlystopper = EarlyStopping(monitor='val_loss', patience=2, verbose=1, restore_best_weights=True)
reducel = ReduceLROnPlateau(monitor='val_loss', patience=1, verbose=1, factor=0.1)
history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=13,
                   callbacks=[reducel, earlystopper])


AttributeError: 'Sequential' object has no attribute 'fit_generator'

In [10]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

# make a prediction
y_pred_keras = model.predict_generator(test_gen, steps=len(df_val), verbose=1)


AttributeError: 'Sequential' object has no attribute 'predict_generator'

In [11]:
base_test_dir = '../input/test_images/'

test_files = glob(os.path.join(base_test_dir,'*.png'))

os.makedirs('valid/')

for image in tqdm(test_files):
    fname = image.split('/')[-1]
    
    src = os.path.join(base_test_dir, fname)
    dst = os.path.join("valid/",fname)
    
    pil_im = Image.open(src)
    resized_image = pil_im.resize((IMAGE_SIZE, IMAGE_SIZE))
    resized_image.save(dst)
    
test_files = glob(os.path.join('valid','*.png'))


submission = pd.DataFrame()
file_batch = 20
max_idx = len(test_files)
for idx in range(0, max_idx, file_batch):
    print("Indexes: %i - %i"%(idx, idx+file_batch))
    test_df = pd.DataFrame({'path': test_files[idx:idx+file_batch]})
    test_df['id_code'] = test_df.path.map(lambda x: x.split('/')[1].split(".")[0])
    test_df['image'] = test_df['path'].map(imread)
    K_test = np.stack(test_df["image"].values)
    K_test = (K_test - K_test.mean()) / K_test.std()
    predictions = model.predict(K_test)
    
    pred = []
    
    for l in predictions:
        pred.append(np.argmax(l))
    
    
    test_df['diagnosis'] = pred
    submission = pd.concat([submission, test_df[["id_code", "diagnosis"]]])
submission.head()


100%|██████████| 367/367 [00:30<00:00, 12.07it/s]


Indexes: 0 - 20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
Indexes: 20 - 40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Indexes: 40 - 60
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Indexes: 60 - 80
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Indexes: 80 - 100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Indexes: 100 - 120
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Indexes: 120 - 140
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Indexes: 140 - 160
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Indexes: 160 - 180
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Indexes: 180 - 200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Indexes: 200 - 220
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Indexes: 220 - 240
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Indexes: 240 - 260
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Indexes: 260 - 280
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Indexes: 280 - 300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Indexes: 300 - 320
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Indexes: 320 - 340
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Indexes: 340 - 360
1/1 ━━━━━━━━━━━━━━━━━

,id_code,diagnosis
0,f9156aeffc5e,2
1,28824d12d31d,2
2,4c60b10a3a6a,3
3,6cbc3dad809c,3
4,1cb814ed6332,3


In [12]:
#submission
# Delete the test_dir directory we created to prevent a Kaggle error.
# Kaggle allows a max of 500 files to be saved.

shutil.rmtree(train_path)
shutil.rmtree(valid_path)
shutil.rmtree('valid/')
submission.to_csv("submission.csv", index = False, header = True)


In [13]:
df = pd.read_csv("submission.csv")
df["diagnosis"].value_counts()


diagnosis
3    261
2     64
4     29
1     12
0      1
Name: count, dtype: int64